In [51]:
import random as rn
import numpy as np

from cfair.backends import NumpyBackend
from cfair.metrics.kernel.hgr import CategoricalHGR, DoubleKernelHGR

In [2]:
backend = NumpyBackend()

# OHE kernel function

In [3]:
def one_hot_encode(x):
        unique_vals = np.unique(x)
        return np.array([[1 if val == xi else 0 for val in unique_vals] for xi in x]).transpose()

# Toy Class

In [4]:
toy_one_hot_hgr = CategoricalHGR(
    backend=backend,
    method='trust-constr',
    maxiter=1000,
    eps=1e-9,
    tol=1e-9,
    use_lstsq=True,
    delta_independent=None
)

## First Example - corr = 0

In [5]:
# Example categorical data
a = np.array(['cat', 'dog', 'cat', 'bird'])
b = np.array(['red', 'red', 'blue', 'green'])

In [6]:

ohe_a = one_hot_encode(a)
ohe_b = one_hot_encode(b)

In [7]:
print(ohe_a)

[[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]]


In [8]:
print(ohe_b)

[[0 0 1 0]
 [0 0 0 1]
 [1 1 0 0]]


In [9]:
result = toy_one_hot_hgr._compute(a, b)
print(result)

None


In [10]:
corr, alpha, beta = toy_one_hot_hgr._indicator(ohe_a, ohe_b, None, None)
print(corr, alpha, beta)

DEBUG: inside indicator - f_numpy = [[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]]
DEBUG: inside indicator - g_numpy = [[0 0 1 0]
 [0 0 0 1]
 [1 1 0 0]]
DEBUG: somehow i managed to concatenate them
DEBUG: inside indicator - fg_numpy = [[ 0  0  0  1  0  0 -1  0]
 [ 1  0  1  0  0  0  0 -1]
 [ 0  1  0  0 -1 -1  0  0]]
0.9449111800932964 [0.19385645 0.54457232 0.19385645 0.30321213] [0.22038587 0.22038587 0.24541706 0.3138112 ]


## Second Example - corr = 0

In [11]:
c = np.array(['cat', 'dog', 'cat', 'bird'])
d = np.array(['cat', 'dog', 'cat', 'bird'])

In [12]:
ohe_c = one_hot_encode(c)
ohe_d = one_hot_encode(d)

In [13]:
print(ohe_c)

[[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]]


In [14]:
print(ohe_d)

[[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]]


In [15]:
result = toy_one_hot_hgr._compute(c, d)
print(result)

None


In [16]:
corr, alpha, beta = toy_one_hot_hgr._indicator(ohe_c, ohe_d, None, None)
print(corr, alpha, beta)

DEBUG: inside indicator - f_numpy = [[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]]
DEBUG: inside indicator - g_numpy = [[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]]
DEBUG: somehow i managed to concatenate them
DEBUG: inside indicator - fg_numpy = [[ 0  0  0  1  0  0  0 -1]
 [ 1  0  1  0 -1  0 -1  0]
 [ 0  1  0  0  0 -1  0  0]]
0.999999998875 [0.26516504 0.26516504 0.26516504 0.26516504] [0.25 0.25 0.25 0.25]


# Polinomial Kernels test

In [17]:
polynomial_kernel = DoubleKernelHGR(
    backend=backend,          
    kernel_a=2, 
    kernel_b=3, 
)

## Joke Experiment

In [18]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([5, 4, 3, 2, 1])

result_p = polynomial_kernel._compute(a, b)

DEBUG: inside result - f = [array([1, 2, 3, 4, 5], dtype=int32), array([ 1,  4,  9, 16, 25])]
DEBUG: inside result - f len = 2
DEBUG: inside result - g = [array([5, 4, 3, 2, 1], dtype=int32), array([25, 16,  9,  4,  1]), array([125,  64,  27,   8,   1], dtype=int32)]
DEBUG: inside result - g len = 3
DEBUG: inside result - f after strange things = [[ -2. -10.]
 [ -1.  -7.]
 [  0.  -2.]
 [  1.   5.]
 [  2.  14.]]
DEBUG: inside result - f len = 5
DEBUG: inside result - g after strange things = [[  2.  14.  80.]
 [  1.   5.  19.]
 [  0.  -2. -18.]
 [ -1.  -7. -37.]
 [ -2. -10. -44.]]
DEBUG: inside result - g len = 5
DEBUG: inside indicator - f_numpy = [[ -2. -10.]
 [ -1.  -7.]
 [  0.  -2.]
 [  1.   5.]
 [  2.  14.]]
DEBUG: inside indicator - g_numpy = [[  2.  14.  80.]
 [  1.   5.  19.]
 [  0.  -2. -18.]
 [ -1.  -7. -37.]
 [ -2. -10. -44.]]
DEBUG: somehow i managed to concatenate them
DEBUG: inside indicator - fg_numpy = [[ -2. -10.  -2. -14. -80.]
 [ -1.  -7.  -1.  -5. -19.]
 [  0.  -2.  

In [19]:
print(result_p.value)

0.9999999989999997


In [20]:
print(result_p.alpha)

[-0.98971955 -0.01028045]


In [21]:
print(result_p.beta)

[ 9.90848562e-01 -9.15143279e-03 -5.25302806e-09]


## 1000, all random

In [66]:
num1 = []
num2 = []

for i in range(1000):
    num1.append(rn.randint(0,9))
    num2.append(rn.randint(0,9))

In [67]:
result_1000_pol = polynomial_kernel._compute(num1, num2)

DEBUG: inside result - f = [array([5, 5, 1, 9, 3, 7, 6, 2, 1, 4, 5, 0, 7, 2, 6, 4, 6, 5, 1, 0, 1, 7,
       0, 0, 1, 6, 7, 0, 4, 3, 5, 9, 1, 9, 6, 0, 3, 0, 8, 2, 3, 9, 4, 1,
       3, 9, 6, 8, 2, 9, 0, 1, 4, 9, 5, 7, 0, 3, 6, 0, 9, 8, 6, 9, 1, 1,
       0, 1, 0, 5, 3, 9, 5, 7, 0, 5, 9, 6, 6, 2, 3, 4, 3, 4, 6, 8, 5, 8,
       3, 4, 1, 3, 6, 6, 1, 8, 1, 6, 2, 0, 1, 5, 6, 0, 3, 0, 0, 7, 0, 3,
       0, 7, 9, 6, 4, 9, 3, 5, 8, 5, 7, 6, 2, 4, 1, 8, 3, 8, 6, 5, 1, 7,
       0, 8, 4, 2, 7, 3, 5, 6, 4, 1, 8, 6, 4, 5, 2, 5, 5, 2, 7, 9, 0, 3,
       5, 1, 4, 2, 4, 0, 3, 6, 1, 6, 6, 2, 3, 8, 3, 6, 1, 2, 8, 3, 2, 5,
       7, 1, 4, 1, 4, 4, 5, 3, 9, 1, 7, 2, 8, 2, 5, 7, 2, 9, 5, 4, 1, 6,
       5, 9, 2, 3, 5, 1, 1, 5, 4, 1, 8, 3, 7, 6, 0, 0, 9, 7, 0, 2, 6, 1,
       6, 7, 6, 6, 5, 3, 9, 2, 5, 2, 1, 0, 1, 8, 5, 6, 5, 4, 7, 0, 4, 4,
       4, 1, 4, 8, 1, 4, 5, 2, 6, 0, 7, 0, 0, 5, 4, 0, 9, 1, 9, 8, 9, 8,
       1, 7, 4, 1, 5, 3, 8, 1, 5, 0, 9, 4, 7, 2, 3, 5, 5, 4, 9, 3, 5, 6,
       5, 7, 0, 8, 4, 1

In [68]:
print(result_1000_pol.value)

0.08017720643133372


In [69]:
print(result_1000_pol.alpha)

[-0.9747018  0.0252982]


In [70]:
print(result_1000_pol.beta)

[ 0.81339446 -0.17539198  0.01121356]


# Instantiating my class

In [22]:
my_kernel_one_hot = DoubleKernelHGR(
    backend=backend,          # Or any other appropriate backend
    kernel_a=one_hot_encode, # Custom kernel function for variable a
    kernel_b=one_hot_encode, # Custom kernel function for variable b
)

## Experiment 1 (numbers)

In [23]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([5, 4, 2, 2, 1])

# Compute the HGR indicator
result1 = my_kernel_one_hot._compute(a, b)

DEBUG: inside result - f = [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
DEBUG: inside result - f len = 5
DEBUG: inside result - g = [[0 0 0 0 1]
 [0 0 1 1 0]
 [0 1 0 0 0]
 [1 0 0 0 0]]
DEBUG: inside result - g len = 4
DEBUG: inside result - f after strange things = [[ 0.8 -0.2 -0.2 -0.2 -0.2]
 [-0.2  0.8 -0.2 -0.2 -0.2]
 [-0.2 -0.2  0.8 -0.2 -0.2]
 [-0.2 -0.2 -0.2  0.8 -0.2]
 [-0.2 -0.2 -0.2 -0.2  0.8]]
DEBUG: inside result - f len = 5
DEBUG: inside result - g after strange things = [[-0.2 -0.4 -0.2  0.8]
 [-0.2 -0.4  0.8 -0.2]
 [-0.2  0.6 -0.2 -0.2]
 [-0.2  0.6 -0.2 -0.2]
 [ 0.8 -0.4 -0.2 -0.2]]
DEBUG: inside result - g len = 5
DEBUG: inside indicator - f_numpy = [[ 0.8 -0.2 -0.2 -0.2 -0.2]
 [-0.2  0.8 -0.2 -0.2 -0.2]
 [-0.2 -0.2  0.8 -0.2 -0.2]
 [-0.2 -0.2 -0.2  0.8 -0.2]
 [-0.2 -0.2 -0.2 -0.2  0.8]]
DEBUG: inside indicator - g_numpy = [[-0.2 -0.4 -0.2  0.8]
 [-0.2 -0.4  0.8 -0.2]
 [-0.2  0.6 -0.2 -0.2]
 [-0.2  0.6 -0.2 -0.2]
 [ 0.8 -0.4 -0.2 -0.2]]
DEBUG: someho

In [24]:
print(result1.value)

0.9999999990000001


In [25]:
print(result1.alpha)

[0.20000184 0.20000837 0.20000051 0.20000051 0.19998878]


In [26]:
print(result1.beta)

[0.24998613 0.25000079 0.25001062 0.25000246]


## Experiment 2 (categorical)

In [34]:
a2 = np.array(['cat', 'dog', 'cat', 'dog'])
b2 = np.array(['sus', 'sis', 'sus', 'ses'])

result2 = my_kernel_one_hot._compute(a2, b2)

DEBUG: inside result - f = [[1 0 1 0]
 [0 1 0 1]]
DEBUG: inside result - f len = 2
DEBUG: inside result - g = [[0 0 0 1]
 [0 1 0 0]
 [1 0 1 0]]
DEBUG: inside result - g len = 3
DEBUG: inside result - f after strange things = [[ 0.5 -0.5]
 [-0.5  0.5]
 [ 0.5 -0.5]
 [-0.5  0.5]]
DEBUG: inside result - f len = 4
DEBUG: inside result - g after strange things = [[-0.25 -0.25  0.5 ]
 [-0.25  0.75 -0.5 ]
 [-0.25 -0.25  0.5 ]
 [ 0.75 -0.25 -0.5 ]]
DEBUG: inside result - g len = 4
DEBUG: inside indicator - f_numpy = [[ 0.5 -0.5]
 [-0.5  0.5]
 [ 0.5 -0.5]
 [-0.5  0.5]]
DEBUG: inside indicator - g_numpy = [[-0.25 -0.25  0.5 ]
 [-0.25  0.75 -0.5 ]
 [-0.25 -0.25  0.5 ]
 [ 0.75 -0.25 -0.5 ]]
DEBUG: somehow i managed to concatenate them
DEBUG: inside indicator - fg_numpy = [[ 0.5  -0.5   0.25  0.25 -0.5 ]
 [-0.5   0.5   0.25 -0.75  0.5 ]
 [ 0.5  -0.5   0.25  0.25 -0.5 ]
 [-0.5   0.5  -0.75  0.25  0.5 ]]


In [35]:
print(result2.value)

0.9999999989999999


In [36]:
print(result2.a)

['cat' 'dog' 'cat' 'dog']


In [37]:
print(result2.b)

['sus' 'sis' 'sus' 'ses']


In [38]:
print(result2.alpha)

[0.49998419 0.50001581]


In [39]:
print(result2.beta)

[0.33334036 0.33334036 0.33331928]


## DEBUG CONCATENATION

In [40]:
f = [ [1, 0],[0, 1],[1, 0],[1, 0] ]

f = np.stack(f, axis=1)
f = f - np.mean(f, axis=0)

print(f)

# g = np.stack(g, axis=1)
# g = g - np.mean(g, axis=0)

# print(g)

[[ 0.5 -0.5  0.5  0.5]
 [-0.5  0.5 -0.5 -0.5]]


## 1000 len experiment EXTRAVAGANZA

### All Random

In [95]:
warriors = ["Ultramarine", "Salamander", "White Scar", "Space Wolf", "Raven Guard", "Iron Hand", "Imperial Fist", "Blood Angel", "Dark Angel"]
animals = ["cat", "dog", "monke", "horse"]

def sample(a_list):
    return rn.choice(a_list)

ani = []
war = []

for i in range(1000):
    ani.append(sample(animals))
    war.append(sample(warriors))

In [96]:
print(ani)

['horse', 'horse', 'monke', 'horse', 'horse', 'dog', 'cat', 'horse', 'monke', 'horse', 'cat', 'cat', 'monke', 'cat', 'monke', 'dog', 'monke', 'monke', 'dog', 'horse', 'horse', 'horse', 'dog', 'dog', 'cat', 'dog', 'monke', 'horse', 'dog', 'horse', 'dog', 'dog', 'monke', 'horse', 'dog', 'horse', 'dog', 'monke', 'monke', 'dog', 'cat', 'monke', 'horse', 'horse', 'horse', 'cat', 'dog', 'cat', 'cat', 'horse', 'cat', 'horse', 'horse', 'monke', 'dog', 'monke', 'monke', 'dog', 'cat', 'horse', 'monke', 'dog', 'monke', 'horse', 'monke', 'cat', 'cat', 'cat', 'monke', 'monke', 'dog', 'monke', 'monke', 'monke', 'monke', 'cat', 'horse', 'dog', 'horse', 'horse', 'monke', 'horse', 'monke', 'monke', 'cat', 'horse', 'horse', 'cat', 'monke', 'monke', 'cat', 'monke', 'dog', 'horse', 'monke', 'horse', 'horse', 'monke', 'cat', 'monke', 'monke', 'horse', 'dog', 'horse', 'horse', 'dog', 'horse', 'cat', 'monke', 'monke', 'horse', 'horse', 'monke', 'horse', 'monke', 'horse', 'monke', 'cat', 'cat', 'dog', 'horse'

In [97]:
print(war)

['Imperial Fist', 'Salamander', 'Dark Angel', 'Ultramarine', 'Ultramarine', 'Raven Guard', 'Imperial Fist', 'Blood Angel', 'Raven Guard', 'Salamander', 'Raven Guard', 'Dark Angel', 'Blood Angel', 'Raven Guard', 'White Scar', 'White Scar', 'Raven Guard', 'Space Wolf', 'Raven Guard', 'Space Wolf', 'Raven Guard', 'Imperial Fist', 'White Scar', 'Imperial Fist', 'White Scar', 'Salamander', 'White Scar', 'Space Wolf', 'Salamander', 'Dark Angel', 'Dark Angel', 'Space Wolf', 'Dark Angel', 'Iron Hand', 'Imperial Fist', 'Space Wolf', 'Ultramarine', 'White Scar', 'Blood Angel', 'Salamander', 'Iron Hand', 'Space Wolf', 'Imperial Fist', 'Iron Hand', 'Space Wolf', 'White Scar', 'Space Wolf', 'Space Wolf', 'White Scar', 'Raven Guard', 'Dark Angel', 'Dark Angel', 'Raven Guard', 'Iron Hand', 'Dark Angel', 'Space Wolf', 'Iron Hand', 'Dark Angel', 'Blood Angel', 'Space Wolf', 'Blood Angel', 'Raven Guard', 'Ultramarine', 'Blood Angel', 'Blood Angel', 'Imperial Fist', 'White Scar', 'Space Wolf', 'Salamande

In [98]:
result_1000 = my_kernel_one_hot._compute(ani, war)

DEBUG: inside result - f = [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 1 0 ... 0 0 0]
 [0 0 1 ... 1 1 0]]
DEBUG: inside result - f len = 4
DEBUG: inside result - g = [[0 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
DEBUG: inside result - g len = 9
DEBUG: inside result - f after strange things = [[-0.243 -0.239  0.74  -0.258]
 [-0.243 -0.239  0.74  -0.258]
 [-0.243 -0.239 -0.26   0.742]
 ...
 [-0.243 -0.239 -0.26   0.742]
 [-0.243 -0.239 -0.26   0.742]
 [-0.243  0.761 -0.26  -0.258]]
DEBUG: inside result - f len = 1000
DEBUG: inside result - g after strange things = [[-0.098 -0.111  0.887 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 ...
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106  0.887]
 [-0.098 -0.111 -0.113 ...  0.88  -0.106 -0.113]]
DEBUG: inside result - g len = 1000
DEBUG: inside indicator

In [99]:
print(result_1000.value)

0.11939975841313756


In [100]:
print(result_1000.alpha)

[0.25000121 0.2499992  0.24999896 0.25000063]


In [101]:
print(result_1000.beta)

[0.11111431 0.11111071 0.11110994 0.11111351 0.11110822 0.11110943
 0.11111008 0.1111055  0.11111829]


### Autocorrelation

In [102]:
result_1000_auto = my_kernel_one_hot._compute(war, war)

DEBUG: inside result - f = [[0 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
DEBUG: inside result - f len = 9
DEBUG: inside result - g = [[0 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
DEBUG: inside result - g len = 9
DEBUG: inside result - f after strange things = [[-0.098 -0.111  0.887 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 ...
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106  0.887]
 [-0.098 -0.111 -0.113 ...  0.88  -0.106 -0.113]]
DEBUG: inside result - f len = 1000
DEBUG: inside result - g after strange things = [[-0.098 -0.111  0.887 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 ...
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 [-0.09

In [103]:
print(result_1000_auto.value)

0.9999999990000064


In [104]:
print(result_1000_auto.alpha)

[0.11111149 0.11111296 0.11111606 0.11111414 0.11110932 0.11111009
 0.11110642 0.11110512 0.1111144 ]


In [105]:
print(result_1000_auto.beta)

[0.11111149 0.11111296 0.11111606 0.11111414 0.11110932 0.11111009
 0.11110642 0.11110512 0.1111144 ]


It is VERY HIGH -> as i expected, VERY GOODDE!!

### "Via di mezzo"

In [112]:
ani2 = []

for warrior in war:
    if warrior == "White Scar":
        print("For the Khan!")
        anim = "horse"
    else:
        anim = sample(animals)
    ani2.append(anim)

For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For the Khan!
For th

In [113]:
print(ani2)

['horse', 'horse', 'horse', 'horse', 'monke', 'horse', 'monke', 'dog', 'horse', 'cat', 'dog', 'cat', 'horse', 'cat', 'horse', 'horse', 'horse', 'cat', 'cat', 'monke', 'monke', 'horse', 'horse', 'cat', 'horse', 'dog', 'horse', 'horse', 'monke', 'monke', 'cat', 'monke', 'monke', 'monke', 'cat', 'dog', 'monke', 'horse', 'cat', 'horse', 'horse', 'cat', 'monke', 'monke', 'monke', 'horse', 'monke', 'horse', 'horse', 'monke', 'cat', 'horse', 'dog', 'horse', 'dog', 'dog', 'cat', 'cat', 'dog', 'monke', 'horse', 'monke', 'monke', 'horse', 'monke', 'horse', 'horse', 'dog', 'cat', 'dog', 'horse', 'horse', 'monke', 'dog', 'cat', 'horse', 'cat', 'monke', 'monke', 'monke', 'dog', 'monke', 'horse', 'horse', 'dog', 'dog', 'cat', 'horse', 'cat', 'monke', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'horse', 'horse', 'monke', 'dog', 'monke', 'monke', 'horse', 'dog', 'horse', 'monke', 'monke', 'monke', 'horse', 'monke', 'horse', 'cat', 'monke', 'monke', 'cat', 'horse', 'dog', 'horse', 'horse', 'monke', 'monk

In [122]:
result_1000_via_di_mezzo = my_kernel_one_hot._compute(war, ani2)

DEBUG: inside result - f = [[0 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
DEBUG: inside result - f len = 9
DEBUG: inside result - g = [[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
DEBUG: inside result - g len = 4
DEBUG: inside result - f after strange things = [[-0.098 -0.111  0.887 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 ...
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106  0.887]
 [-0.098 -0.111 -0.113 ...  0.88  -0.106 -0.113]]
DEBUG: inside result - f len = 1000
DEBUG: inside result - g after strange things = [[-0.197 -0.226  0.673 -0.25 ]
 [-0.197 -0.226  0.673 -0.25 ]
 [-0.197 -0.226  0.673 -0.25 ]
 ...
 [ 0.803 -0.226 -0.327 -0.25 ]
 [-0.197 -0.226  0.673 -0.25 ]
 [-0.197 -0.226 -0.327  0.75 ]]
DEBUG: inside result - g len = 1000
DEBUG: inside indicator

In [123]:
print(result_1000_via_di_mezzo.value)

0.5174255822215944


In [124]:
print(result_1000_via_di_mezzo.alpha)

[0.11111058 0.11111056 0.11111046 0.11110984 0.1111107  0.11111066
 0.11111073 0.1111103  0.11111616]


In [125]:
print(result_1000_via_di_mezzo.beta)

[0.24999579 0.24999577 0.25001264 0.24999579]


# Calling directly "_result" [very important experiment, discovered a LOT of things]

If we do not initialize a0 and b0, they will NOT change. BUT, if we initialize them to be all ones, well they will indeed change and at the end their value will highlight the most correlated categories. This is awesome!!

In [139]:
result_1000_via_di_mezzo = my_kernel_one_hot._result(war, ani2, kernel_a=True, kernel_b=True, a0=np.ones(9), b0=np.ones(4))

DEBUG: inside result - f = [[0 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
DEBUG: inside result - f len = 9
DEBUG: inside result - g = [[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
DEBUG: inside result - g len = 4
DEBUG: inside result - f after strange things = [[-0.098 -0.111  0.887 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 ...
 [-0.098  0.889 -0.113 ... -0.12  -0.106 -0.113]
 [-0.098 -0.111 -0.113 ... -0.12  -0.106  0.887]
 [-0.098 -0.111 -0.113 ...  0.88  -0.106 -0.113]]
DEBUG: inside result - f len = 1000
DEBUG: inside result - g after strange things = [[-0.197 -0.226  0.673 -0.25 ]
 [-0.197 -0.226  0.673 -0.25 ]
 [-0.197 -0.226  0.673 -0.25 ]
 ...
 [ 0.803 -0.226 -0.327 -0.25 ]
 [-0.197 -0.226  0.673 -0.25 ]
 [-0.197 -0.226 -0.327  0.75 ]]
DEBUG: inside result - g len = 1000
DEBUG: inside indicator

In [140]:
print(result_1000_via_di_mezzo.value)

0.5174255822242023


In [141]:
print(result_1000_via_di_mezzo.alpha)

[0.09439636 0.09373294 0.09058542 0.07105265 0.09817142 0.09673061
 0.0991237  0.08537309 0.27083381]


In [142]:
print(np.unique(result_1000_via_di_mezzo.a))

['Blood Angel' 'Dark Angel' 'Imperial Fist' 'Iron Hand' 'Raven Guard'
 'Salamander' 'Space Wolf' 'Ultramarine' 'White Scar']


In [143]:
print(result_1000_via_di_mezzo.a)

['Imperial Fist', 'Salamander', 'Dark Angel', 'Ultramarine', 'Ultramarine', 'Raven Guard', 'Imperial Fist', 'Blood Angel', 'Raven Guard', 'Salamander', 'Raven Guard', 'Dark Angel', 'Blood Angel', 'Raven Guard', 'White Scar', 'White Scar', 'Raven Guard', 'Space Wolf', 'Raven Guard', 'Space Wolf', 'Raven Guard', 'Imperial Fist', 'White Scar', 'Imperial Fist', 'White Scar', 'Salamander', 'White Scar', 'Space Wolf', 'Salamander', 'Dark Angel', 'Dark Angel', 'Space Wolf', 'Dark Angel', 'Iron Hand', 'Imperial Fist', 'Space Wolf', 'Ultramarine', 'White Scar', 'Blood Angel', 'Salamander', 'Iron Hand', 'Space Wolf', 'Imperial Fist', 'Iron Hand', 'Space Wolf', 'White Scar', 'Space Wolf', 'Space Wolf', 'White Scar', 'Raven Guard', 'Dark Angel', 'Dark Angel', 'Raven Guard', 'Iron Hand', 'Dark Angel', 'Space Wolf', 'Iron Hand', 'Dark Angel', 'Blood Angel', 'Space Wolf', 'Blood Angel', 'Raven Guard', 'Ultramarine', 'Blood Angel', 'Blood Angel', 'Imperial Fist', 'White Scar', 'Space Wolf', 'Salamande

In [144]:
print(result_1000_via_di_mezzo.beta)

[0.06590305 0.06509403 0.80302527 0.06597765]


In [145]:
print(np.unique(result_1000_via_di_mezzo.b))

['cat' 'dog' 'horse' 'monke']


In [146]:
print(result_1000_via_di_mezzo.b)

['horse', 'horse', 'horse', 'horse', 'monke', 'horse', 'monke', 'dog', 'horse', 'cat', 'dog', 'cat', 'horse', 'cat', 'horse', 'horse', 'horse', 'cat', 'cat', 'monke', 'monke', 'horse', 'horse', 'cat', 'horse', 'dog', 'horse', 'horse', 'monke', 'monke', 'cat', 'monke', 'monke', 'monke', 'cat', 'dog', 'monke', 'horse', 'cat', 'horse', 'horse', 'cat', 'monke', 'monke', 'monke', 'horse', 'monke', 'horse', 'horse', 'monke', 'cat', 'horse', 'dog', 'horse', 'dog', 'dog', 'cat', 'cat', 'dog', 'monke', 'horse', 'monke', 'monke', 'horse', 'monke', 'horse', 'horse', 'dog', 'cat', 'dog', 'horse', 'horse', 'monke', 'dog', 'cat', 'horse', 'cat', 'monke', 'monke', 'monke', 'dog', 'monke', 'horse', 'horse', 'dog', 'dog', 'cat', 'horse', 'cat', 'monke', 'dog', 'dog', 'cat', 'cat', 'cat', 'dog', 'horse', 'horse', 'monke', 'dog', 'monke', 'monke', 'horse', 'dog', 'horse', 'monke', 'monke', 'monke', 'horse', 'monke', 'horse', 'cat', 'monke', 'monke', 'cat', 'horse', 'dog', 'horse', 'horse', 'monke', 'monk